In [1]:
from sklearn import datasets
import pandas as pd
import numpy as np

iris = datasets.load_iris()  # датасет
iris_frame = pd.DataFrame(iris.data)  # создаем таблицу
iris_frame.columns = iris.feature_names  # подписываем столбцы

# для удобства добавис столбец с целевой переменной, где 0 - setosa, 1 - versicolor, 2 - virginica
iris_frame['target'] = iris.target
# к нему же добавим столбец с сортом ириса для наглядности
iris_frame['name'] = iris_frame.target.apply(lambda x: iris.target_names[x])
iris_frame

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,name
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2,virginica
146,6.3,2.5,5.0,1.9,2,virginica
147,6.5,3.0,5.2,2.0,2,virginica
148,6.2,3.4,5.4,2.3,2,virginica


In [2]:
alpha = 0.00005

#создаем слои, задаем начальные веса
weight_1_2 = np.random.randn(4, 4)
weight_2_3 = np.random.randn(4, 4)
weight_3_4 = np.random.randn(4, 4)
weight_4_5 = np.random.randn(4, 1)

print(weight_1_2, weight_2_3, weight_3_4, weight_4_5, sep="\n\n")

[[ 0.29206166  0.38174769  0.87189041 -0.15388659]
 [-0.44951816  0.11438988  1.1792291  -0.22231231]
 [ 0.01029593 -2.18808532 -0.15598437 -0.13779536]
 [ 0.31353375  1.17084197 -0.14876245  1.2967325 ]]

[[-0.39318527  1.46489191  0.30127467  0.03134649]
 [ 0.12011685  0.91281311  0.82085068 -0.36651253]
 [-0.23438797  0.65777634 -0.65571832 -1.04720603]
 [ 0.43353867  0.87698161  0.71877109 -0.53922993]]

[[-0.21861612 -1.10224882  0.01169095  1.64825204]
 [ 0.94502973 -0.20221157  0.11147647  0.60135688]
 [ 0.20851621 -0.26375302  0.81498468 -0.33189055]
 [ 0.25394264  0.48777034 -0.33927122 -1.20612574]]

[[-0.02128815]
 [-0.17331088]
 [-0.8329503 ]
 [-0.24674165]]


In [3]:
# -------обучение------ методом обратного распространения ошибки

for iteration in range(250):
    for i in range(len(iris_frame["sepal length (cm)"])):
        inp = [iris_frame["sepal length (cm)"][i], iris_frame["sepal width (cm)"][i],
               iris_frame["petal length (cm)"][i], iris_frame["petal width (cm)"][i]]
        goal_pred = iris_frame["target"][i]  #  берем за эталон таргетное значение
        
        #  вычисляем предсказание нейросети
        layer_2 = np.dot(inp, weight_1_2)
        layer_3 = np.dot(layer_2, weight_2_3)
        layer_4 = np.dot(layer_3, weight_3_4)
        pred = np.sum(np.dot(layer_4, weight_4_5))
        if iteration % 100 == 0:  # на вывод идет только 3 итерации обучения
            if pred <= 0.5:       # для остальных только ошибка
                print("Это - setosa", goal_pred, pred)
            elif pred > 0.7 and pred <= 1.5:
                print("Это - versicolor", goal_pred, pred)
            elif pred > 1.7 and pred <= 2.5:
                print("Это - virginica", goal_pred, pred)
            else:
                print("Я не знаю", goal_pred, pred)
        
        # обратное распространение ошибки (градиентный спуск)
        error = (pred - goal_pred) ** 2   # общая ошибка
        layer_5_delta = pred - goal_pred  # дельты для каждого слоя
        layer_4_delta = np.sum(np.dot(layer_5_delta, weight_4_5))
        layer_3_delta = np.sum(np.dot(layer_4_delta, weight_3_4))
        layer_2_delta = np.sum(np.dot(layer_3_delta, weight_2_3))

        # вычисляем дельты весов
        weight_delta_1_2 = np.zeros(weight_1_2.shape)
        weight_delta_2_3 = np.zeros(weight_2_3.shape)
        weight_delta_3_4 = np.zeros(weight_3_4.shape)
        weight_delta_4_5 = np.zeros(weight_4_5.shape)

        # корректируем веса
        for k in range(len(weight_delta_1_2)):
            for j in range(len(weight_delta_1_2[k])):
                weight_delta_1_2[k][j] = inp[k] * layer_2_delta   

        for k in range(len(weight_delta_2_3)):
            for j in range(len(weight_delta_2_3[k])):
                weight_delta_2_3[k][j] = np.sum(layer_2.T[j]) * layer_3_delta

        for k in range(len(weight_delta_3_4)):
            for j in range(len(weight_delta_3_4[k])):
                weight_delta_3_4[k][j] = np.sum(layer_3.T[j]) * layer_4_delta

        for k in range(len(weight_delta_4_5)):
            weight_delta_4_5[k] = np.sum(layer_4.T[k]) * layer_5_delta


        for k in range(len(weight_1_2)):
            for j in range(len(weight_1_2)):
                weight_1_2[k][j] -= weight_delta_1_2[k][j] * alpha

        for k in range(len(weight_2_3)):
            for j in range(len(weight_2_3)):
                weight_2_3[k][j] -= weight_delta_2_3[k][j] * alpha

        for k in range(len(weight_3_4)):
            for j in range(len(weight_3_4)):
                weight_3_4[k][j] -= weight_delta_3_4[k][j] * alpha

        for k in range(len(weight_4_5)):     
            weight_4_5[k] -= weight_delta_4_5[k] * alpha
            
    print(iteration, error, sep=" iteration, error: ")  # отслеживаем ошибку по мере обучения

Это - setosa 0 -0.19451254545578767
Это - setosa 0 -0.08486040542263806
Это - setosa 0 -0.1853358989229672
Это - setosa 0 0.285656913979323
Это - setosa 0 -0.13510958931576678
Это - setosa 0 -0.18293208847375286
Это - setosa 0 -0.15178178420389887
Это - setosa 0 0.09058917816286982
Это - setosa 0 0.18520789764486034
Это - setosa 0 0.3653395498053187
Это - setosa 0 -0.13877718921622728
Это - setosa 0 0.38186328213508736
Это - setosa 0 0.18800430941672297
Это - setosa 0 -0.18044309875450026
Это - setosa 0 -0.9753535537441933
Это - setosa 0 -0.7184895678373948
Это - setosa 0 -0.9437717180457608
Это - setosa 0 -0.3124697044461273
Это - setosa 0 0.025852352237134557
Это - setosa 0 -0.08924936882734169
Это - setosa 0 0.40623726136901617
Это - setosa 0 -0.3391355225511452
Это - setosa 0 -0.6407827780646376
Это - setosa 0 -0.12695478287862105
Это - versicolor 0 1.1445996499941704
Это - setosa 0 0.3743122036491777
Это - setosa 0 -0.10274936905339516
Это - setosa 0 0.06428258011875787
Это - seto

70 iteration, error: 0.03151254042413751
71 iteration, error: 0.032128937357474054
72 iteration, error: 0.03273388833256738
73 iteration, error: 0.033327365280541504
74 iteration, error: 0.03390935846070068
75 iteration, error: 0.034479875231529136
76 iteration, error: 0.03503893888080618
77 iteration, error: 0.035586587511386394
78 iteration, error: 0.0361228729794917
79 iteration, error: 0.036647859882769085
80 iteration, error: 0.03716162459561338
81 iteration, error: 0.03766425434954937
82 iteration, error: 0.038155846356666884
83 iteration, error: 0.03863650697430802
84 iteration, error: 0.039106350909405445
85 iteration, error: 0.039565500460928896
86 iteration, error: 0.040014084799152264
87 iteration, error: 0.040452239280401765
88 iteration, error: 0.04088010479617484
89 iteration, error: 0.041297827155487965
90 iteration, error: 0.04170555649942012
91 iteration, error: 0.04210344674685779
92 iteration, error: 0.04249165507047572
93 iteration, error: 0.042870341402009314
94 it

137 iteration, error: 0.05250212357230641
138 iteration, error: 0.05260861082280266
139 iteration, error: 0.052711779041038256
140 iteration, error: 0.05281170946992735
141 iteration, error: 0.05290848126683405
142 iteration, error: 0.05300217154974382
143 iteration, error: 0.05309285544351468
144 iteration, error: 0.05318060612608926
145 iteration, error: 0.05326549487456077
146 iteration, error: 0.053347591110976214
147 iteration, error: 0.05342696244779398
148 iteration, error: 0.05350367473292931
149 iteration, error: 0.05357779209427246
150 iteration, error: 0.05364937698368028
151 iteration, error: 0.05371849022032217
152 iteration, error: 0.05378519103336518
153 iteration, error: 0.05384953710395748
154 iteration, error: 0.053911584606466396
155 iteration, error: 0.053971388248929136
156 iteration, error: 0.05402900131273202
157 iteration, error: 0.054084475691435416
158 iteration, error: 0.05413786192879665
159 iteration, error: 0.05418920925592266
160 iteration, error: 0.05423

205 iteration, error: 0.05498453983777381
206 iteration, error: 0.0549780813693779
207 iteration, error: 0.054970936838239115
208 iteration, error: 0.054963121159321414
209 iteration, error: 0.05495464891971384
210 iteration, error: 0.054945534386959864
211 iteration, error: 0.05493579151715872
212 iteration, error: 0.05492543396281434
213 iteration, error: 0.05491447508047502
214 iteration, error: 0.0549029279381404
215 iteration, error: 0.054890805322452794
216 iteration, error: 0.05487811974570296
217 iteration, error: 0.054864883452597
218 iteration, error: 0.05485110842687736
219 iteration, error: 0.05483680639772054
220 iteration, error: 0.05482198884596062
221 iteration, error: 0.05480666701016184
222 iteration, error: 0.05479085189248286
223 iteration, error: 0.054774554264396806
224 iteration, error: 0.05475778467226263
225 iteration, error: 0.05474055344271302
226 iteration, error: 0.05472287068792689
227 iteration, error: 0.054704746310710324
228 iteration, error: 0.05468619

In [5]:
# применяем выборку на обученной НС

for i in range(len(iris_frame["sepal length (cm)"])):
        inp = [iris_frame["sepal length (cm)"][i], iris_frame["sepal width (cm)"][i],
               iris_frame["petal length (cm)"][i], iris_frame["petal width (cm)"][i]]
        goal_pred = iris_frame["target"][i]
        
        layer_2 = np.dot(inp, weight_1_2)
        layer_3 = np.dot(layer_2, weight_2_3)
        layer_4 = np.dot(layer_3, weight_3_4)
        pred = np.sum(np.dot(layer_4, weight_4_5))
        if pred <= 0.5:
            print(i, "Это - setosa", goal_pred, pred)
        elif pred > 0.5 and pred <= 1.7:
            print(i, "Это - versicolor", goal_pred, pred)
        elif pred > 1.7:
            print(i, "Это - virginica", goal_pred, pred)
        else:
            print(i, "Я не знаю", goal_pred, pred)

0 Это - setosa 0 -0.03264445713195413
1 Это - setosa 0 0.003979753153291354
2 Это - setosa 0 -0.029709129516059862
3 Это - setosa 0 0.13037264656197456
4 Это - setosa 0 -0.014074145693088791
5 Это - setosa 0 0.04113338881139317
6 Это - setosa 0 0.01800638621049172
7 Это - setosa 0 0.05660781339899773
8 Это - setosa 0 0.0963886709826311
9 Это - setosa 0 0.116641106414766
10 Это - setosa 0 -0.017157019763978543
11 Это - setosa 0 0.16443039536881593
12 Это - setosa 0 0.06423436585155323
13 Это - setosa 0 -0.055771459732855355
14 Это - setosa 0 -0.30341036956820266
15 Это - setosa 0 -0.15564637077938492
16 Это - setosa 0 -0.24218463337693663
17 Это - setosa 0 -0.07440253161884913
18 Это - setosa 0 0.027328075436687305
19 Это - setosa 0 -0.003351706389271847
20 Это - setosa 0 0.12428067164769191
21 Это - setosa 0 -0.045183554103665036
22 Это - setosa 0 -0.22340601503594687
23 Это - setosa 0 0.05442228959361256
24 Это - setosa 0 0.37691891201006344
25 Это - setosa 0 0.12714222603608832
26 Эт